# Search Log analyser
Provides top 10 search results

## TODO:

1. [x] show top 20 results
2. [ ] add top 10 by month
3. [x] add profanity check 
4. [ ] by year

In [288]:
import pandas as pd
import numpy as np

search_logs = 'search-logs.log'

In [289]:
with open(search_logs, 'r') as file:
    split_list = [line.strip() for line in file.readlines()]

# Create a DataFrame
df = pd.DataFrame(split_list, columns=["log_entry"])


In [290]:
# Split each line by the colon ":"
df_split = df["log_entry"].str.split(":", expand=True)

# Display the first few rows
# df_split.head()

In [291]:
df = df_split.drop(columns=[0,1,3,4,5,7,8,9])
# df.head()

In [292]:
df.rename(columns={2:'date',6:'term'}, inplace=True)
# df.head()

In [293]:
log_data = df
# log_data.head()

In [294]:
print(log_data.dtypes)
#log_data['date'] = pd.to_datetime(log_data['date'])

# log_data.head(10)

date    object
term    object
dtype: object


In [295]:
# Clean up data
log_data['term'] = log_data['term'].str.strip()
log_data['term'] = log_data['term'].str.upper()
log_data['term'] = log_data['term'].replace('"','').replace("'","")

# Remove null and empty and quote makrs

log_data.head()





,date,term
0,2024-01-07T13,PYTHON
1,2024-01-07T13,PYTHON
2,2024-01-07T13,ESP32
3,2024-01-07T13,ESP32
4,2024-01-07T13,ESP32


In [296]:

log_data = log_data[~log_data['term'].str.contains('NULL',case=False,na=False)]
log_data = log_data[~log_data['term'].str.contains('""',case=False,na=False)]

In [297]:
term_counts = log_data.groupby('term').size().reset_index(name='count')

# Display the result
print(term_counts)


                             term  count
0                                    154
1     ! PIP INSTALL NUMPY, PANDAS      5
2                     "BURGERBOT"      1
3                           "CAR"      2
4                        "EVENTS"      1
...                           ...    ...
1713                         ЗМЕЯ      1
1714                          משד      1
1715                          رشس      1
1716                    سةشقس ؤشق      1
1717                      شقيعهىخ      1

[1718 rows x 2 columns]


In [298]:
# Sort by the 'count' column in descending order
term_counts_sorted = term_counts.sort_values(by='count', ascending=False)

# Display the sorted result
print(term_counts_sorted)

         term  count
1311      ROS    399
343    DOCKER    175
0                154
1163   PYTHON    142
921    OLLAMA    127
...       ...    ...
681      JOIN      1
680      JEYK      1
677   JENKINS      1
674      JEKY      1
1717  شقيعهىخ      1

[1718 rows x 2 columns]


In [204]:
df = term_counts_sorted[~term_counts_sorted['term'].str.strip().eq('')]
df = df[~df['term'].str.contains('NULL',case=False,na=False)]

df.head(20)

,term,count
0,"! PIP INSTALL NUMPY, PANDAS",1
1154,PSP,1
1152,PROXIMITY,1
1151,PROTOCOL FIRMATA + CODR ARDUINO,1
1150,PROBLEMS WITH BRACHIOGRAPH,1
1149,PRIVATE AI REVOLUTION,1
1148,PRINTING,1
1147,PRINTER,1
1146,PRINTABLE,1
1145,PRINT,1


In [205]:
# Check for profanity
with open("profanity.txt", "r", encoding="utf-8") as f:
    profane_words = [line.strip().lower() for line in f if line.strip()]

# Convert terms to lowercase
log_data["lower_term"] = log_data["term"].str.lower()

# Create a boolean column indicating if the term is in the list of profane words
log_data["is_profane"] = log_data["lower_term"].isin(profane_words)

filtered_df = log_data[log_data["is_profane"] == True]
print(filtered_df)

#print(log_data)


Empty DataFrame
Columns: [term, count, lower_term, is_profane]
Index: []


In [193]:
## Top search Results per month
df['date'] = pd.to_datetime(df["date"])

df["year_month"] = df["date"].dt.to_period("M")
grouped = (
    df.groupby(["year_month", "term"])
      .size()  # Count occurrences
      .reset_index(name="count")
)
print(grouped)
grouped_sorted = grouped.sort_values(["year_month", "count"], ascending=[True, False])
top_5_each_month = grouped_sorted.groupby("year_month").head(5)
print(top_5_each_month)


KeyError: 'date'